In [16]:
import zipfile
import os
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [17]:
# Descomprimir el archivo ZIP
zip_file_path = 'Elements.zip'
extract_dir = 'Elements'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [18]:
# Configuración de los parámetros
batch_size = 32
img_height = 150
img_width = 150
num_classes = len(os.listdir(extract_dir))  # Número de clases

In [19]:
# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # 20% para validación
)

In [20]:
# Cargar los datos
train_data_gen = datagen.flow_from_directory(
    extract_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data_gen = datagen.flow_from_directory(
    extract_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 180 images belonging to 30 classes.
Found 30 images belonging to 30 classes.


In [21]:
# Definir la arquitectura del modelo
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])


In [22]:
# Compilar el modelo
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
# Entrenar el modelo
epochs = 10  # Puedes ajustar el número de épocas
history = model.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // batch_size,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // batch_size,
    epochs=epochs
)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 404ms/step - accuracy: 0.0613 - loss: 3.4077 - val_accuracy: 0.3333 - val_loss: 3.0949
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.1250 - loss: 3.2158 - val_accuracy: 0.5000 - val_loss: 3.0265
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.2367 - loss: 2.8393 - val_accuracy: 0.6000 - val_loss: 1.5682
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.2500 - loss: 2.2447 - val_accuracy: 0.7333 - val_loss: 1.2447
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.4780 - loss: 1.6469 - val_accuracy: 0.9000 - val_loss: 0.5068
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6562 - loss: 1.5077 - val_accuracy: 0.9667 - val_loss: 0.4114
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.7916 - loss: 0.7217 - val_accuracy: 1.0000 - val_loss: 0.1085
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7188 - loss: 0.7968 - val_accuracy: 1.0000 - val_loss: 0.0873

In [24]:
# Guardar el modelo
model.save('modelo_caracteres.h5')